# Getting Started with FinGPT
Welcome to this comprehensive guide aimed at beginners diving into the realm of Financial Large Language Models (FinLLMs) with FinGPT. This blog post demystifies the process of training FinGPT using Low-Rank Adaptation (LoRA) with the robust base model ChatGlm2-6b.



## Part 1: Preparing the Data
Data preparation is a crucial step when it comes to training Financial Large Language Models. Here, we’ll guide you on how to get your dataset ready for FinGPT using Python.

In this section, you’ve initialized your working directory and loaded a financial sentiment dataset. Let’s break down the steps:



In [1]:
!pip install datasets transformers torch tqdm pandas huggingface_hub
!pip install sentencepiece
!pip install protobuf transformers==4.30.2 cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate
!pip install torchvision torchaudio
!pip install loguru
!pip install datasets
!pip install peft
!pip install tensorboard

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [51 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
  creating build\lib.win-amd64-cpython-312\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
  creating build\lib.win-amd64-cpython-312\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
  creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-31

   ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
   ------------------- -------------------- 30.7/62.5 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 62.5/62.5 kB 670.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/5.5 MB 656.4 kB/s eta 0:00:09
   ---------------------------------------- 0.1/5.5 MB 656.4 kB/s eta 0:00:09
    --------------------------------------- 0.1/5.5 MB 476.3 kB/s eta 0:00:12
    --------------------------------------- 0.1/5.5 MB 504.4 kB/s eta 0:00:11
    --------------------------------------- 0.1/5.5 MB 450.6 kB/s eta 0:00:12
   - -------------------------------------- 0.1/5.5 MB 448.2 kB/s eta 0:00:12
   - ---------------------

### 1.1 Initialize Directories:
This block checks if certain paths exist; if they do, it deletes them to avoid data conflicts, and then creates a new directory for the upcoming data.



In [1]:
import os
import shutil

jsonl_path = "../data/dataset_new.jsonl"
save_path = '../data/dataset_new'


if os.path.exists(jsonl_path):
    os.remove(jsonl_path)

if os.path.exists(save_path):
    shutil.rmtree(save_path)

directory = "../data"
if not os.path.exists(directory):
    os.makedirs(directory)


### 1.2 Load and Prepare Dataset:

* Import necessary libraries from the datasets package: https://huggingface.co/docs/datasets/index
* Load the Twitter Financial News Sentiment (TFNS) dataset and convert it to a Pandas dataframe. https://huggingface.co/datasets/zeroshot/twitter-financial-news-sentiment
* Map numerical labels to their corresponding sentiments (negative, positive, neutral).
* Add instruction for each data entry, which is crucial for Instruction Tuning.
* Convert the Pandas dataframe back to a Hugging Face Dataset object.




In [2]:
from datasets import load_dataset
import datasets

dic = {
    0:"negative",
    1:'positive',
    2:'neutral',
}

tfns = load_dataset('zeroshot/twitter-financial-news-sentiment')
tfns = tfns['train']
tfns = tfns.to_pandas()
tfns['label'] = tfns['label'].apply(lambda x:dic[x])
tfns['instruction'] = 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.'
tfns.columns = ['input', 'output', 'instruction']
tfns = datasets.Dataset.from_pandas(tfns)
tfns

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9543
})

### 1.3 Concatenate and Shuffle Dataset

In [3]:
tmp_dataset = datasets.concatenate_datasets([tfns]*2)
train_dataset = tmp_dataset
print(tmp_dataset.num_rows)

all_dataset = train_dataset.shuffle(seed = 42)
all_dataset.shape

19086


(19086, 3)

Now that your training data is loaded and prepared.

## Part 2: Dataset Formatting and Tokenization
Once your data is prepared, the next steps involve formatting the dataset for model ingestion and tokenizing the input data. Below, we provide a step-by-step breakdown of the code snippets shared.



### 2.1 Dataset Formatting:
You need to structure your data in a specific format that aligns with the training process.



In [4]:
import json
from tqdm.notebook import tqdm

In [5]:
def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

In [6]:
data_list = []
for item in all_dataset.to_pandas().itertuples():
    tmp = {}
    tmp["instruction"] = item.instruction
    tmp["input"] = item.input
    tmp["output"] = item.output
    data_list.append(tmp)

In [7]:
# save to a jsonl file
with open("../data/dataset_new.jsonl", 'w') as f:
    for example in tqdm(data_list, desc="formatting.."):
        f.write(json.dumps(format_example(example)) + '\n')

formatting..:   0%|          | 0/19086 [00:00<?, ?it/s]

### 2.2 Tokenization
Tokenization is the process of converting input text into tokens that can be fed into the model.



In [8]:
import datasets
from transformers import AutoTokenizer, AutoConfig

model_name = "THUDM/chatglm2-6b"
jsonl_path = "../data/dataset_new.jsonl"  # updated path
save_path = '../data/dataset_new'  # updated path
max_seq_length = 512
skip_overlength = True

In [9]:
# The preprocess function tokenizes the prompt and target, combines them into input IDs,
# and then trims or pads the sequence to the maximum sequence length.
def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

# The read_jsonl function reads each line from the JSONL file, preprocesses it using the preprocess function,
# and then yields each preprocessed example.
def read_jsonl(path, max_seq_length, skip_overlength=False):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

### 2.3 Save the dataset

In [10]:
# The script then creates a Hugging Face Dataset object from the generator and saves it to disk.
save_path = '../data/dataset_new'

dataset = datasets.Dataset.from_generator(
    lambda: read_jsonl(jsonl_path, max_seq_length, skip_overlength)
    )
dataset.save_to_disk(save_path)


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

C:\Users\F21Z\anaconda3\envs\for_fingpt\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\F21Z\.cache\huggingface\hub\models--THUDM--chatglm2-6b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenization_chatglm.py:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

configuration_chatglm.py:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


  0%|          | 0/19086 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/19086 [00:00<?, ? examples/s]

## Part 3: Setup FinGPT training parameters with LoRA on ChatGlm2-6b

Training a model can be resource-intensive. Ensure you have a powerful GPU
 * Need to purchase Google Colab GPU plans, Colab Pro is sufficient or just buy 100 compute units for $10
 * NVIDIA A100 is recommended due to its high memory capacity.



In [11]:
# Ensure CUDA is accessible in the system path
# Only for Windows Subsystem for Linux (WSL)
import os
os.environ["PATH"] = f"{os.environ['PATH']}:/usr/local/cuda/bin"
os.environ['LD_LIBRARY_PATH'] = "/usr/lib/wsl/lib:/usr/local/cuda/lib64"

In [12]:
pip install --upgrade peft


Note: you may need to restart the kernel to use updated packages.


### 3.1 Training Arguments Setup:
Initialize and set training arguments.



In [13]:
from typing import List, Dict, Optional
import torch
from loguru import logger
from transformers import (
    AutoModel,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import (
    TaskType,
    LoraConfig,
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

In [14]:
training_args = TrainingArguments(
        output_dir='./finetuned_model',    # saved model path
        logging_steps = 500,
        # max_steps=10000,
        num_train_epochs = 2,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        learning_rate=1e-4,
        weight_decay=0.01,
        warmup_steps=1000,
        save_steps=500,
        fp16=True,
        # bf16=True,
        torch_compile = False,
        load_best_model_at_end = True,
        evaluation_strategy="steps",
        remove_unused_columns=False,

    )

C:\Users\F21Z\anaconda3\envs\for_fingpt\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
!pip list | grep bitsandbytes

'grep' is not recognized as an internal or external command,
operable program or batch file.


### 3.2 Quantization Config Setup:
Set quantization configuration to reduce model size without losing significant precision.



In [16]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [17]:
 # Quantization
q_config = BitsAndBytesConfig(load_in_4bit=False,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_compute_dtype=torch.float16
                                )

In [ ]:
!pip install accelerate

In [18]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


### 3.3 Model Loading & Preparation:
Load the base model and tokenizer, and prepare the model for INT8 training.

* **Runtime -> Change runtime type -> A100 GPU**
* retart runtime and run again if not working


In [19]:
# Load tokenizer & model
# need massive space
model_name = "THUDM/chatglm2-6b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
        model_name,
        quantization_config=q_config,
        trust_remote_code=True,
        device='cuda'
    )
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

modeling_chatglm.py:   0%|          | 0.00/54.9k [00:00<?, ?B/s]

quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

...

pytorch_model-00004-of-00007.bin:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 214.00 MiB. GPU 

### 3.4 LoRA Config & Setup:
Implement Low-Rank Adaptation (LoRA) and print trainable parameters.



In [20]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [21]:
# LoRA
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules,
    bias='none',
)
model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

NameError: name 'model' is not defined

In [22]:
!python -m bitsandbytes


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++ OTHER +++++++++++++++++++++++++++
CUDA specs: CUDASpecs(highest_compute_capability=(8, 9), cuda_version_string='121', cuda_version_tuple=(12, 1))
PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: (8, 9).
To manually override the PyTorch CUDA version please see: https://github.com/TimDettmers/bitsandbytes/blob/main/docs/source/nonpytorchcuda.mdx
CUDA SETUP: WARNING! CUDA runtime files not found in any environmental path.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++ DEBUG INFO END ++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking that the library is importable and CUDA is callable...
SUCCESS!
Installation was successful!


The directory listed in your path is found to be non-existent: \\LAPTOP-TMR2PPV4
C:\Users\F21Z\anaconda3\envs\for_fingpt\Lib\site-packages\bitsandbytes\diagnostics\main.py:33: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  p = torch.nn.Parameter(torch.rand(10, 10).cuda())


In [23]:
resume_from_checkpoint = None
if resume_from_checkpoint is not None:
    checkpoint_name = os.path.join(resume_from_checkpoint, 'pytorch_model.bin')
    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, 'adapter_model.bin'
        )
        resume_from_checkpoint = False
    if os.path.exists(checkpoint_name):
        logger.info(f'Restarting from {checkpoint_name}')
        adapters_weights = torch.load(checkpoint_name)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        logger.info(f'Checkpoint {checkpoint_name} not found')

In [24]:
model.print_trainable_parameters()

NameError: name 'model' is not defined

## Part 4: Loading Data and Training FinGPT
In this segment, we'll delve into the loading of your pre-processed data, and finally, launch the training of your FinGPT model. Here's a stepwise breakdown of the script provided:
* Need to purchase Google Colab GPU plans, Colab Pro is sufficient or just buy 100 compute units for $10


### 4.1 Loading Your Data:


In [25]:
# load data
from datasets import load_from_disk
import datasets

dataset = datasets.load_from_disk("../data/dataset_new")
dataset = dataset.train_test_split(0.2, shuffle=True, seed = 42)

### 4.2 Training Configuration and Launch:
* Customize the Trainer class for specific loss computation, prediction step, and model-saving methods.

* Define a data collator function to process batches of data during training.

* Set up TensorBoard for logging, instantiate your modified trainer, and begin training.



In [26]:
class ModifiedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss

    def prediction_step(self, model: torch.nn.Module, inputs, prediction_loss_only: bool, ignore_keys = None):
        with torch.no_grad():
            res = model(
                input_ids=inputs["input_ids"].to(model.device),
                labels=inputs["labels"].to(model.device),
            ).loss
        return (res, None, None)

    def save_model(self, output_dir=None, _internal_call=False):
        from transformers.trainer import TRAINING_ARGS_NAME

        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        saved_params = {
            k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
        }
        torch.save(saved_params, os.path.join(output_dir, "adapter_model.bin"))

def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [tokenizer.pad_token_id] * (seq_len - 1) + ids[(seq_len - 1) :] + [tokenizer.pad_token_id] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

In [27]:
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback

In [28]:
# Train
# Took about 10 compute units
writer = SummaryWriter()
trainer = ModifiedTrainer(
    model=model,
    args=training_args,             # Trainer args
    train_dataset=dataset["train"], # Training set
    eval_dataset=dataset["test"],   # Testing set
    data_collator=data_collator,    # Data Collator
    callbacks=[TensorBoardCallback(writer)],
)
trainer.train()
writer.close()
# save model
model.save_pretrained(training_args.output_dir)

NameError: name 'model' is not defined

### 4.3 Model Saving and Download:
After training, save and download your model. You can also check the model's size.



In [29]:
!zip -r /content/saved_model.zip /content/{training_args.output_dir}


'zip' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
# download to local
from google.colab import files
files.download('/content/saved_model.zip')

ModuleNotFoundError: No module named 'google.colab'

In [31]:
# save to google drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [32]:
# save the finetuned model to google drive
!cp -r "/content/finetuned_model" "/content/drive/MyDrive"


'cp' is not recognized as an internal or external command,
operable program or batch file.


In [33]:
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, _, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / 1024 / 1024  # Size in MB

model_size = get_folder_size(training_args.output_dir)
print(f"Model size: {model_size} MB")


Model size: 0.0 MB


Now your model is trained and saved! You can download it and use it for generating financial insights or any other relevant tasks in the finance domain. The usage of TensorBoard allows you to deeply understand and visualize the training dynamics and performance of your model in real-time.

Happy FinGPT Training! 🚀

## Part 5: Inference and Benchmarks using FinGPT
Now that your model is trained, let’s understand how to use it to infer and run benchmarks.
* Took about 10 compute units



In [34]:
!pip install transformers==4.30.2 peft==0.4.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes

  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
   ---------------------------------------- 0.0/72.9 kB ? eta -:--:--
   ----------- ---------------------------- 20.5/72.9 kB ? eta -:--:--
   ---------------------------- ----------- 51.2/72.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 72.9/72.9 kB 1.0 MB/s eta 0:00:00
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [51 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
  creating build\lib.win-amd64-cpython-312\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
  creating build\lib.win-amd64-cpython-312\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
  creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-31

### 5.1 Load the model

In [35]:
#clone the FinNLP repository
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git


import sys
sys.path.append('/content/FinNLP/')

'git' is not recognized as an internal or external command,
operable program or batch file.


In [36]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

from peft import PeftModel
import torch

# Load benchmark datasets from FinNLP
from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

ModuleNotFoundError: No module named 'finnlp'

In [37]:
!pip install --upgrade peft

In [38]:
# load model from google drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [39]:
# Define the path you want to check
path_to_check = "/content/drive/My Drive/finetuned_model"

# Check if the specified path exists
if os.path.exists(path_to_check):
    print("Path exists.")
else:
    print("Path does not exist.")


Path does not exist.


In [40]:
from transformers import AutoModelForSequenceClassification


In [41]:
## load the chatglm2-6b base model
base_model = "THUDM/chatglm2-6b"
peft_model = training_args.output_dir

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModel.from_pretrained(base_model, trust_remote_code=True, load_in_8bit=True, device_map = "cuda:0")

model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


KeyboardInterrupt: 

In [ ]:
## load our finetuned model
base_model = "THUDM/chatglm2-6b"
peft_model = "/content/drive/My Drive/finetuned_model"

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModel.from_pretrained(base_model, trust_remote_code=True, load_in_8bit=True, device_map="auto")

model = PeftModel.from_pretrained(model, peft_model)

model = model.eval()


### 5.2 Run Benchmarks:

In [ ]:
batch_size = 8

In [ ]:
# TFNS Test Set, len 2388
# Available: 84.85 compute units
res = test_tfns(model, tokenizer, batch_size = batch_size)
# Available: 83.75 compute units
# Took about 1 compute unite to inference



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 8. Total steps: 299


100%|██████████| 299/299 [01:42<00:00,  2.91it/s]

Acc: 0.859715242881072. F1 macro: 0.8052933389732111. F1 micro: 0.859715242881072. F1 weighted (BloombergGPT): 0.8524619717597879. 


In [ ]:
# FPB, len 1212
res = test_fpb(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 8. Total steps: 152


100%|██████████| 152/152 [00:52<00:00,  2.88it/s]

Acc: 0.7747524752475248. F1 macro: 0.7145731809778274. F1 micro: 0.7747524752475248. F1 weighted (BloombergGPT): 0.7509437786642208. 


In [ ]:
# FiQA, len 275
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 8. Total steps: 35


100%|██████████| 35/35 [00:12<00:00,  2.69it/s]

Acc: 0.45454545454545453. F1 macro: 0.46471842589900164. F1 micro: 0.45454545454545453. F1 weighted (BloombergGPT): 0.5417737714451875. 


In [ ]:
# NWGI, len 4047
res = test_nwgi(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/16184 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4047 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 8. Total steps: 506


100%|██████████| 506/506 [03:00<00:00,  2.81it/s]

Acc: 0.5495428712626637. F1 macro: 0.5322328963435276. F1 micro: 0.5495428712626637. F1 weighted (BloombergGPT): 0.5347497805673743. 


### 5.3 Compare it with FinGPT V3.1 results
https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT-*v3*

#### Comparison


***TFNS:***

***FinGPT V3.1:***

* Acc: 0.876
* F1 macro: 0.841
* F1 weighted (follow BloombergGPT): 0.875

**This notebook**:
* Acc: 0.856
* F1 macro: 0.806
* F1 weighted (follow BloombergGPT): 0.850

#### Since we trained on the TFNS dataset, it is expected that the test results would be good.
----
***FPB:***

***FinGPT V3.1:***
* Acc: 0.856
* F1 macro: 0.841
* F1 weighted: 0.855

**This notebook**:
* Acc: 0.741
* F1 macro: 0.655
* F1 weighted: 0.694

#### Considering the FPB dataset was not included in our training set, the obtained zero-shot results are acceptable.


----
***FiQA:***

***FinGPT V3.1:***
* Acc: 0.836
* F1 macro: 0.746
* F1 weighted: 0.850

**This notebook**:
* Acc: 0.48
* F1 macro: 0.5
* F1 weighted: 0.49

#### Since the FiQA dataset wasn't part of our training set, our model's zero-shot performance is relatively poor compared to FinGPT V3.1.

----
***NWGI:***

***FinGPT V3.1:***
* Acc: 0.642
* F1 macro: 0.650
* F1 weighted: 0.642

**This notebook**:
* Acc: 0.521
* F1 macro: 0.500
* F1 weighted: 0.490

#### The results are reasonable

## Conclusion:

* The training and testing of FinGPT in this exercise demanded **a total of 20 compute units**, broken down into 10 for training and another 10 for inference.
* 100 Compute Units for 10 dollars, that **makes 2 dollars to train and test with FinGPT**
* This cost-effective approach is primarily attributable to the utilization of the **Low-Rank Adaptation (LoRA) method**, which proves to be economical while ensuring efficient model training and inference.



This exercise provided insights into the performance of your trained FinGPT model across various benchmarks. While there are areas where it excels, certain benchmarks highlight opportunities for improvement and tuning. Exploring additional training data and refining the model further will likely lead to enhanced performance across different financial NLP tasks, making it a powerful tool for various applications in the finance sector.



Happy Experimenting with FinGPT! 🚀



